# Prepare Datset AirBNB Dataset New York

### Imports

In [94]:
import pandas as pd

### Read Detailed Listing Data from last 4 Quaters

In [95]:
# Combine quarterly NY Listing Data
df1 = pd.read_csv(
    "../Data/listings20220603.csv.gz",
    compression="gzip",
    parse_dates=["first_review", "last_review", "host_since", "calendar_last_scraped"],
    dtype={
        "bathrooms_text": str,
    },
)
df1["Quarter"] = 20220603
df2 = pd.read_csv(
    "../Data/listings20220907.csv.gz",
    compression="gzip",
    parse_dates=["first_review", "last_review", "host_since", "calendar_last_scraped"],
    dtype={
        "bathrooms_text": str,
    },
)
df2["Quarter"] = 20220907
df3 = pd.read_csv(
    "../Data/listings20221204.csv.gz",
    compression="gzip",
    parse_dates=["first_review", "last_review", "host_since", "calendar_last_scraped"],
    dtype={
        "bathrooms_text": str,
    },
)
df3["Quarter"] = 20221204
df4 = pd.read_csv(
    "../Data/listings20230306.csv.gz",
    compression="gzip",
    parse_dates=["first_review", "last_review", "host_since", "calendar_last_scraped"],
    dtype={
        "bathrooms_text": str,
    },
)
df4["Quarter"] = 20230306
nyc_data = pd.concat([df1, df2, df3, df4])

C:\Users\Felix\AppData\Local\Temp\ipykernel_6112\1684282793.py:2: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(
C:\Users\Felix\AppData\Local\Temp\ipykernel_6112\1684282793.py:11: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(
C:\Users\Felix\AppData\Local\Temp\ipykernel_6112\1684282793.py:20: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv(
C:\Users\Felix\AppData\Local\Temp\ipykernel_6112\1684282793.py:29: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv(


## Preprocessing

### Removing irrelevant Columns

In [96]:
# percentage of values that are missing
total_nan = nyc_data.isna().sum().sort_values(ascending=False)
percentage_nan = (total_nan / nyc_data.shape[0]) * 100
tabel = pd.concat([total_nan, percentage_nan], axis=1, keys=['Total NaN values', 'Percentage of NaN values']).head(60)
tabel

,Total NaN values,Percentage of NaN values
calendar_updated,161755,100.000000
bathrooms,161755,100.000000
license,161744,99.993200
host_about,71712,44.333715
neighborhood_overview,66900,41.358845
neighbourhood,66896,41.356372
host_response_rate,54935,33.961856
host_response_time,54935,33.961856
host_acceptance_rate,48368,29.902012
review_scores_location,37861,23.406386


In [97]:
drop_cols = [
    "bathrooms", #Empty Column
    "calendar_updated", #Empty Column
    "license", #Almost Empty Column
    "name", # textual description already extracted as continous variables in other columns
    "listing_url", #listing_url is unique and adds no value to the model
    "scrape_id",#scape_id is unique and adds no value to the model
    "last_scraped",#last_scaped is unique and adds no value to the model
    "picture_url", #picture_url adds no value to the model
    "host_url", #host_url adds no value to the model
    "host_about", #host_about adds no value to the model
    "host_thumbnail_url", #host_thumbnail adds no value to the model
    "host_picture_url", #host_picture_url adds no value to the model
    "host_location", #Same Information as neighbourhood_cleansed. Keep only neighbourhood_cleansed because it has no NAN Values
    "host_neighbourhood", #Same Information as neighbourhood_cleansed. Keep only neighbourhood_cleansed because it has no NAN Values
    "neighbourhood", #Same Information as neighbourhood_cleansed. Keep only neighbourhood_cleansed because it has no NAN Values
    "neighborhood_overview", # Text information diffivult preprocessing
    "description", # Text information diffivult preprocessing
    "host_name", # 
    "amenities", #Text information diffivult preprocessing
    "source" #Not relevant

]
# making a new dataframe
airbnb = nyc_data.drop(drop_cols, axis=1)
airbnb.shape

(161755, 56)

### Split bathroom text

In [98]:
def split_bathroom_text(text):
    text = str(text).lower()
    try:
        number = float(str(text).split()[0])
    except ValueError:
        number = None
    
    if "shared" in text:
        bathroom_type = "shared"
    else:
        bathroom_type = "private"
    return pd.Series([number, bathroom_type])

airbnb[["bathroom_number", "bathroom_type"]] = airbnb["bathrooms_text"].apply(split_bathroom_text)
airbnb = airbnb.drop("bathrooms_text", axis=1)

### Fill Nan

In [99]:
airbnb['reviews_per_month'].fillna(airbnb['reviews_per_month'].mean(), inplace=True)
airbnb['host_response_rate'].fillna(airbnb['host_response_rate'].mean(), inplace=True)
airbnb['host_acceptance_rate'].fillna(airbnb['host_acceptance_rate'].mean(), inplace=True)
airbnb['review_scores_value'].fillna(airbnb['review_scores_value'].mean(), inplace=True)
airbnb['review_scores_location'].fillna(airbnb['review_scores_location'].mean(), inplace=True)
airbnb['review_scores_checkin'].fillna(airbnb['review_scores_checkin'].mean(), inplace=True)
airbnb['review_scores_accuracy'].fillna(airbnb['review_scores_accuracy'].mean(), inplace=True)
airbnb['review_scores_cleanliness'].fillna(airbnb['review_scores_cleanliness'].mean(), inplace=True)
airbnb['review_scores_communication'].fillna(airbnb['review_scores_communication'].mean(), inplace=True)
airbnb['review_scores_rating'].fillna(airbnb['review_scores_rating'].mean(), inplace=True)
airbnb['bedrooms'].fillna(1, inplace=True)
airbnb.fillna({'host_is_superhost':'f', 'host_has_profile_pic': 'f', 'host_identity_verified': 'f'}, inplace=True)



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Felix\Documents\Studium\Master_Semester_3\PredictiveDataAnalytics\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Felix\AppData\Local\Temp\ipykernel_6112\805270618.py", line 2, in <module>
    airbnb['host_response_rate'].fillna(airbnb['host_response_rate'].mean(), inplace=True)
                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Felix\Documents\Studium\Master_Semester_3\PredictiveDataAnalytics\.venv\Lib\site-packages\pandas\core\generic.py", line 11556, in mean
    return NDFrame.mean(self, axis, skipna, numeric_only, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Felix\Documents\Studium\Master_Semester_3\PredictiveDataAnalytics\.venv\Lib\site-packages\pandas\core\generic.py", line 11201, in mean
    return self._stat_function(
           ^^^^

### String to float

In [100]:
airbnb["price"]= airbnb["price"].replace('[\$,]', '', regex=True).astype(float)
airbnb["host_response_rate"]= airbnb["host_response_rate"].replace('[\%,]', '', regex=True).astype(float)
airbnb["host_acceptance_rate"]= airbnb["host_acceptance_rate"].replace('[\%,]', '', regex=True).astype(float)
airbnb['host_is_superhost'] = airbnb['host_is_superhost'].map({'t': True, 'f': False})
airbnb['host_has_profile_pic'] = airbnb['host_has_profile_pic'].map({'t': True, 'f': False})
airbnb['host_identity_verified'] = airbnb['host_identity_verified'].map({'t': True, 'f': False})
airbnb['has_availability'] = airbnb['has_availability'].map({'t': True, 'f': False})
airbnb['instant_bookable'] = airbnb['instant_bookable'].map({'t': True, 'f': False})

### Removing Outliers

In [101]:
print(airbnb["price"].describe())
airbnb = airbnb[(airbnb['price'] <= 500) & (airbnb['price'] > 0)]

count    161755.000000
mean        202.986574
std         698.401795
min           0.000000
25%          78.000000
50%         129.000000
75%         210.000000
max       99000.000000
Name: price, dtype: float64


In [102]:
print(airbnb["minimum_nights"].describe())
airbnb = airbnb[airbnb['minimum_nights'] <= 50]

count    153607.000000
mean         19.198214
std          30.866462
min           1.000000
25%           2.000000
50%          20.000000
75%          30.000000
max        1250.000000
Name: minimum_nights, dtype: float64


### Add Special features

In [103]:
import geopandas as gpd
import numpy as np
from scipy.spatial import distance

# Load the GeoJSON data for points of interest and subway stations
points_of_interest = gpd.read_file("../Data/Points Of Interest.geojson")
subway_stations = gpd.read_file("../Data/Subway Stations.geojson")

# Extract the x and y coordinates for points of interest
points_of_interest_coords = np.column_stack((points_of_interest.geometry.x, points_of_interest.geometry.y))

# Extract the x and y coordinates for subway stations
subway_stations_coords = np.column_stack((subway_stations.geometry.x, subway_stations.geometry.y))

# Assuming the airbnb variable exists and contains the necessary data
airbnb_coords = np.array(airbnb[['longitude', 'latitude']])

# Calculate the distances to the nearest points of interest
distances_to_points_of_interest = distance.cdist(airbnb_coords, points_of_interest_coords).min(axis=1)
airbnb["distance_to_nearest_point_of_interest"] = distances_to_points_of_interest

# Calculate the distances to the nearest subway stations
distances_to_subway_stations = distance.cdist(airbnb_coords, subway_stations_coords).min(axis=1)
airbnb["distance_to_nearest_subway_station"] = distances_to_subway_stations


### Drop Nan

In [104]:
print(airbnb.shape)
airbnb = airbnb.dropna()
print(airbnb.shape)

(149756, 59)


(71396, 59)


## Save Preprocessed Data

In [105]:
airbnb.to_csv("../Data/Airbnb_data.csv", index=False)
print(airbnb.shape)

(71396, 59)


### Change to kategorial values

In [106]:
airbnb['host_response_time']= airbnb['host_response_time'].astype("category").cat.codes
airbnb['neighbourhood_cleansed']= airbnb['neighbourhood_cleansed'].astype("category").cat.codes
airbnb['neighbourhood_group_cleansed']= airbnb['neighbourhood_group_cleansed'].astype("category").cat.codes
airbnb['property_type']= airbnb['property_type'].astype("category").cat.codes
airbnb['room_type']= airbnb['room_type'].astype("category").cat.codes
airbnb["host_verifications"]= airbnb["host_verifications"].astype("category").cat.codes
airbnb['bathroom_type']= airbnb['bathroom_type'].astype("category").cat.codes

In [107]:
for column in airbnb.select_dtypes(include=["datetime64[ns]"]).columns:
    airbnb[column] = pd.to_numeric(airbnb[column].astype("datetime64[ns]")) / 10**9

In [108]:
airbnb.to_csv("../Data/Airbnb_data_categorial.csv", index=False)
print(airbnb.shape)

(71396, 59)
